In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc, precision_score, classification_report, confusion_matrix, accuracy_score, recall_score, f1_score
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
import scipy.io as sp
from sklearn.decomposition import PCA
from sklearn.linear_model import Perceptron
from sklearn.calibration import CalibratedClassifierCV
import xlsxwriter
import pandas as pd
from IPython.display import display
from sklearn.model_selection import StratifiedKFold




#Traitement du fichier .mat
cancercell = sp.loadmat('breastw.mat')
X = cancercell['X']
y = cancercell['y']
#print(cancercell)
y=np.reshape(y,(683,))
#fin traitement mat

In [ ]:
penalty = ['l1', 'l2','elasticnet','None']
Max_iter = []
for k in range (1,15):
    Max_iter.append(50*k)
Eta =[]
for k in range (-5,3):
    Eta.append(10**(-k)) 
Alpha = []
for k in range (0,5):
    Alpha.append(10**(-k))

#data = np.array([['penalty','Max_iter','Eta','Alpha','F1 score','recall','precision','accuracy','AUC',]])

skf = StratifiedKFold(n_splits=10,shuffle=True) # équilibrage dataset

for train_index, test_index in skf.split(X,y):
    
    test =False
    X_train, X_test = X[train_index,:], X[test_index,:]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np.ravel(y_train)
    y_test=np.ravel(y_test)

    #Méthode Perceptron simple

    test =False
    
    for k in penalty :
        for i in Max_iter :
            for j in Eta :
                for p in Alpha : 
                    #algo Perceptron
                    model_Percep = Perceptron(penalty=k, max_iter=i, eta0=j, alpha=p)
                    model_Percep.fit(X_train,y_train)
                    y_pred_percep = model_Percep.predict(X_test)
                    
                    #fin algo Perceptron
                    
                    #Scores 
                    F1_Percep = round(f1_score(y_test,y_pred_percep),3)
                    recall_Percep = round(recall_score(y_test,y_pred_percep),3)
                    precision_Percep = round(precision_score(y_test,y_pred_percep,zero_division=0),3)
                    accuracy_Percep = round(accuracy_score(y_test,y_pred_percep),3)
                    
                    #Construction matrice hyperparametres et scores
                    if test == False :
                        matrice_Percep = np.array([[k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep]])
                        test = True
                    else :
                        matrice_Percep = np.append(matrice_Percep,np.array([k,i,j,p,F1_Percep,recall_Percep,precision_Percep,accuracy_Percep]).reshape(1,8),axis=0)


#Dataframe 

df = pd.DataFrame(matrice_Percep, columns=['penalty','Max_iter','Eta','Alpha','F1 score','Recall','Precision','Accuracy'])
df = df.astype({'F1 score': float, 'Recall': float,'Precision': float,'Accuracy': float})
filtered_values_1 = np.where((df['F1 score']>=0.96) & (df['Recall']>=0.925) & (df['Precision']>=0.925) & (df['AUC']>=0.925))
display(df.iloc[filtered_values_1].sort_values(by=['F1 score'], ascending=False))
